In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp500.csv')
#stocks

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#type(data)
#print(data) # Response = 200 means success
#print(data.status_code == 200)

In [5]:
print(data)

{'symbol': 'AMZN', 'companyName': 'Amazon.com Inc.', 'primaryExchange': 'BTL(AT KQNAMLCENAOA/ LGESE)SGR DS', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'ilfcifao', 'close': None, 'closeTime': None, 'closeSource': 'flafioic', 'high': None, 'highTime': 1662301124921, 'highSource': 'm ec irra eitlpEIXe', 'low': None, 'lowTime': 1666804396236, 'lowSource': 'eXmrIt ep arEei lci', 'latestPrice': 3464.9, 'latestSource': 'IEX real time price', 'latestTime': '3:10:56 PM', 'latestUpdate': 1670974306395, 'latestVolume': None, 'iexRealtimePrice': 3462, 'iexRealtimeSize': 26, 'iexLastUpdated': 1690081451769, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 3427.92, 'previousVolume': 2283916, 'change': -53.23, 'changePercent': -0.01528, 'volume': None, 'iexMarketPercent': 0.04580762634173128

In [6]:
price = data['latestPrice']
#print(price)
market_cap = data['marketCap']
#print(market_cap/1000000000000)

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe = pd.DataFrame([[0,0,0,0]],columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
    index=my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AMZN,3464.9,1735089637103,N/A


Looping through the tickers in the list

In [9]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:10]:
#for stock in stocks['Ticker']:
    #print(stock)
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index=my_columns),
        ignore_index=True
        )

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,135.510,41723293212,N/A
1,AAL,20.378,13370889268,N/A
2,AAP,207.890,13459341262,N/A
3,AAPL,136.460,2323235381885,N/A
4,ABBV,113.620,196876993389,N/A
5,ABC,125.440,25754474354,N/A
6,ABMD,344.220,15647588083,N/A
7,ABT,125.900,225190170163,N/A
8,ACN,293.210,200885024370,N/A
9,ADBE,531.269,243576538144,N/A


Using the Batch API Calls to improve the performance. <br>
IEX Cloud limits the batch API calls to 100 tickers per request.

In [11]:
def chunks(lst, n):
    # Yield successive n-sized chunks from lst.
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
#symbol_groups
symbol_strings = []
for i in range(0, len(symbol_groups)):
    #print(i)
    #print(symbol_groups[i])
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    #print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,135.750,42667860531,N/A
1,AAL,20.260,13442847874,N/A
2,AAP,202.310,13544883279,N/A
3,AAPL,133.680,2286673046942,N/A
4,ABBV,114.169,196208213912,N/A
...,...,...,...,...
500,YUM,122.920,35616609987,N/A
501,ZBH,179.000,38326871867,N/A
502,ZBRA,505.970,26793651630,N/A
503,ZION,53.450,8766822985,N/A


In [ ]:
portfolio_size = input('Enter the value of your Portfolio: ')

try:
    value = float(portfolio_size)
    #print(val) 
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your Portfolio: ')
    value = float(portfolio_size)


In [ ]:
position_size = value/len(final_dataframe.index)
#print(position_size)
#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

for i in range(0, len(final_dataframe.index)):
    #print(final_dataframe)
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe


Format the Excel Output

In [ ]:
writer = pd.ExcelWriter('recommended.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'recommended', index=False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [ ]:
# writer.sheets['recommended'].set_column('A:A', 18, string_format)
# writer.sheets['recommended'].set_column('B:B', 18, string_format)
# writer.sheets['recommended'].set_column('C:C', 18, string_format)
# writer.sheets['recommended'].set_column('D:D', 18, string_format)
# writer.save()

writer.sheets['recommended'].write('A1', 'Ticker', string_format)
writer.sheets['recommended'].write('B1', 'Stock Price', dollar_format)
writer.sheets['recommended'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['recommended'].write('D1', 'Number of Shares to Buy', integer_format)

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['recommended'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['recommended'].write(f'{column}1', column_formats[column][0], string_format) 

Save the Excel Output

In [ ]:
writer.save()